In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
path = "/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_100/Boris_100.in"
track_path = "/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_100/MS/TRACKS/test_electrons-tracks.h5"
sim = ou.Simulation(path)
track = ou.OsirisTrackFile(track_path)

path = "/home/exxxx5/Tese/Decks/ComparePushers/weibelBG_B0_1/weibelBG_B0_1.in"
sim_original = ou.Simulation(path)


In [ ]:
# # Create tag_file
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_100/MS/RAW/test_electrons/RAW-test_electrons-000000.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_100/tag_file_osiris_utils.tag", type="all")

In [ ]:
# # Convert track file to the readable old format
# ou.utils.convert_tracks(track_path)

In [ ]:
def plot_trajectory(traj):
    '''
    Function to plot a 3D trajectory over time.
    
    Input:
        - traj: A structured numpy array of shape (num_times,) with named fields "x1", "x2", and "x3".
            numpy.ndarray with dtype containing fields "x1", "x2", "x3"
    
    Output:
        - A 3D plot showing the trajectory.
    
    Example:
        >>> traj = track._data[0, :][["x1", "x2", "x3"]]
        >>> plot_trajectory(traj)
    '''
    
    x, y, z = traj["x1"], traj["x2"], traj["x3"]
    
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', color='black', markersize=4, label='Trajectory')
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='g', s=50, label='Start')
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=50, label='End')
    
    # Labels and title
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')
    ax.set_zlabel(r'$x_3$')
    ax.set_title('Particle trajectory')
    ax.legend()

    return fig, ax

In [ ]:
traj=track.data[500, :][["x1", "x2", "x3"]]


fig, ax = plot_trajectory(traj)

plt.tight_layout()
plt.show()